# Demonstration in using Nearest Neighbor Algorithm to Determine NHL Player Comparables

## Introduction

Since the introduction of the salary cap after the lockout in NHL 2004-2005 season, the challenge of building a team has become even harder. One of the biggest challenges teams face is having to deal with the issue of allocating players' salaries within the constraints added by the salary cap. This, along with the growth of advanced analytics has changed the way the players are evaluated. While the NHL has not gone full moneyball yet, there is a greater emphasis on player evaluation and how these players fall within the salary cap. 

One issue that arises is how are players compared from one another. If a player's contract is up or they want to be traded elsewhere, how do you determine how much they should earn on their next contract? How do you replace their contribution and stay within the salary cap constraints? What players could you potentially use to fill that gap in your roster? Often to solve this issue, the player's comparables (other similar players) are looked at and is the benchmark that is used for comparison.

The comparables can be determine by three methods: the eye-test, player production and, advanced analytics.

The eye-test is a qualitative assessment about how a player plays based on watching their game. Examples would include: their play style, their positioning, their effort on plays, and how they handle adversity.

Production is a set of quantitative measures that record outcomes of their play. Examples include: their point totals, their hits, their faceoff wins, and their +/-. 

Advanced analytics are quantitative measures that look at the statistics of how the players play beyond basic production. They can either be instances that are not recorded in basic production stats or engineered features. The idea is to take into account the role of luck and the unpredictability of outcomes. By improving the underlying statistics, you would improve your odds of winning. Examples of measures include: expected goals for, corsi (metrics for possession), point shares, wins or games above replacement.

All three of these methods have value in determining player comparables and have their advantages and disadvantages. Furthmore the measures you use to compare will give you a different set of comparable players. Add on the fact that if you consider salary negotiations, you have two parties (the player's camp and the team's management) who try to anchor to different extremes for salary values. This makes determining comparables difficult as you have a large number of attributes to consider and different groups pushing to utilize different players to give them the upper hand. One technique that could be used in order to determine an accurate group of comparables is the Nearest Neighbor (kNN) model.

The Nearest Neighbor algorithm is well suited for this application. It can be used in order to take observations with n number of inputs and places them on a plane then compares the distances between the observations. We can use the players as observations with the relevant metrics for comparison as the inputs. This allows us to 'plot' the players and look at the players with the smallest distances between them. We can use sci-kit learn and the Kneighbors method as a tool in order to do this. Using it we can compare players and utilize metrics from: the eye-test, production and advanced analytics.

In order to show this, we will perform a demonstration using player data from the NHL 19 video game.

To get the data, I started a franchise mode, simulated the start of the season and then manually collected the attribute data for the top 250 point scorers. This kept the data to players that are likely NHL regulars and skews it toward offensive players, which ensures we have a group of players that are comparables. This data was then prepared for analysis and run through the kNN model.

## Code

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [2]:
#import the nhl 19 player data
df = pd.read_excel('nhl19.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 31 columns):
First Name             250 non-null object
Last Name              250 non-null object
Position               250 non-null object
Age                    250 non-null int64
Shoots                 250 non-null object
Player Type            250 non-null object
Deking                 250 non-null int64
Hand-Eye               250 non-null int64
Passing                250 non-null int64
Puck Control           250 non-null int64
Discipline             250 non-null int64
Off. Awareness         250 non-null int64
Poise                  250 non-null int64
Slap Shot Accuracy     250 non-null int64
Slap Shot Power        250 non-null int64
Wrist Shot Accuracy    250 non-null int64
Wrist Shot Power       250 non-null int64
Def. Awareness         250 non-null int64
Faceoffs               250 non-null int64
Shot Blocking          250 non-null int64
Stick Checking         250 non-null int64
Accelera

In [3]:
# take a look at the observations
df.head()

,First Name,Last Name,Position,Age,Shoots,Player Type,Deking,Hand-Eye,Passing,Puck Control,...,Acceleration,Agility,Balance,Endurance,Speed,Aggressiveness,Body Checking,Durability,Fighting Skill,Strength
0,Auston,Matthews,C,20,Left,PLY,92,91,92,93,...,89,89,88,87,89,80,85,87,65,88
1,Johnny,Gaudreau,LW,25,Left,SNP,95,95,95,96,...,94,95,84,85,93,80,77,85,65,82
2,Alex,Ovechkin,LW,33,Right,SNP,90,96,90,95,...,88,88,95,88,88,89,91,90,80,94
3,Claude,Giroux,C,30,Right,PLY,93,93,94,94,...,91,91,84,85,91,82,82,85,70,84
4,Jonathan,Huberdeau,LW,25,Right,PLY,90,89,91,91,...,89,89,86,85,89,77,77,85,65,85


In [4]:
# engineer a new feature to act as a unique key
df['Key'] = df['First Name'] + df['Last Name']

In [5]:
# eliminate potential gaps between players name for those with spaces in last name like James van Riemsdyk
df['Key'] = df['Key'].str.replace(' ','')

In [6]:
# check that there are no duplicates
df['Key'].value_counts()

YanniGourde           1
JonathanToews         1
JesperiKotkaniemi     1
OndrejPalat           1
MarkStone             1
JamesNeal             1
FransNielsen          1
SeanMonahan           1
BrendanGallagher      1
BlakeWheeler          1
KyleOkposo            1
LouiEriksson          1
KyleTurris            1
RyanNugent-Hopkins    1
JadenSchwartz         1
JasonSpezza           1
JoakimNordstrom       1
RyanEllis             1
TylerEnnis            1
CaleMakar             1
DougieHamilton        1
TimoMeier             1
ChrisKreider          1
BraydenSchenn         1
DrewDoughty           1
ValtteriFilppula      1
LeonDraisaitl         1
OndrejKase            1
DylanLarkin           1
RyanMcDonagh          1
                     ..
PatrickMarleau        1
MilesWood             1
MattiasEkholm         1
AdamHenrique          1
JakeGuentzel          1
MarkScheifele         1
VladimirTarasenko     1
J.T.Compher           1
MikaelBacklund        1
AnzeKopitar           1
MikkoRantanen   

In [7]:
# drop the names
df.drop(columns=['First Name', 'Last Name'], inplace = True)
df.head()

,Position,Age,Shoots,Player Type,Deking,Hand-Eye,Passing,Puck Control,Discipline,Off. Awareness,...,Agility,Balance,Endurance,Speed,Aggressiveness,Body Checking,Durability,Fighting Skill,Strength,Key
0,C,20,Left,PLY,92,91,92,93,90,92,...,89,88,87,89,80,85,87,65,88,AustonMatthews
1,LW,25,Left,SNP,95,95,95,96,95,94,...,95,84,85,93,80,77,85,65,82,JohnnyGaudreau
2,LW,33,Right,SNP,90,96,90,95,80,96,...,88,95,88,88,89,91,90,80,94,AlexOvechkin
3,C,30,Right,PLY,93,93,94,94,85,93,...,91,84,85,91,82,82,85,70,84,ClaudeGiroux
4,LW,25,Right,PLY,90,89,91,91,82,92,...,89,86,85,89,77,77,85,65,85,JonathanHuberdeau


In [8]:
# set the key as the index
df = df.set_index('Key')
df.head()

,Position,Age,Shoots,Player Type,Deking,Hand-Eye,Passing,Puck Control,Discipline,Off. Awareness,...,Acceleration,Agility,Balance,Endurance,Speed,Aggressiveness,Body Checking,Durability,Fighting Skill,Strength
Key,,,,,,,,,,,,,,,,,,,,,
AustonMatthews,C,20,Left,PLY,92,91,92,93,90,92,...,89,89,88,87,89,80,85,87,65,88
JohnnyGaudreau,LW,25,Left,SNP,95,95,95,96,95,94,...,94,95,84,85,93,80,77,85,65,82
AlexOvechkin,LW,33,Right,SNP,90,96,90,95,80,96,...,88,88,95,88,88,89,91,90,80,94
ClaudeGiroux,C,30,Right,PLY,93,93,94,94,85,93,...,91,91,84,85,91,82,82,85,70,84
JonathanHuberdeau,LW,25,Right,PLY,90,89,91,91,82,92,...,89,89,86,85,89,77,77,85,65,85


In [9]:
# check the different possible values for the position
df['Position'].unique()

array(['C', 'LW', 'RW', 'D'], dtype=object)

In [10]:
# enumerate 
df['Position'] = df['Position'].map({'C':0,'D':1,'RW':2,'LW':3})

In [11]:
# check instances of shooting hand
df['Shoots'].unique()

array(['Left', 'Right'], dtype=object)

In [12]:
# enumerate
df['Shoots'] = df['Shoots'].map({'Left':0,'Right':1})

In [13]:
# check player types
df['Player Type'].unique()

array(['PLY', 'SNP', 'TWF', 'PWF', 'TWD', 'OFD', 'GRN'], dtype=object)

In [14]:
# enumerate
df['Player Type'] = df['Player Type'].map({'PLY':0,'SNP':1,'TWF':2,'PWF':3,'TWD':4,'OFD':5,'GRN':6})

In [15]:
# take a look at the data
df.head()

,Position,Age,Shoots,Player Type,Deking,Hand-Eye,Passing,Puck Control,Discipline,Off. Awareness,...,Acceleration,Agility,Balance,Endurance,Speed,Aggressiveness,Body Checking,Durability,Fighting Skill,Strength
Key,,,,,,,,,,,,,,,,,,,,,
AustonMatthews,0,20,0,0,92,91,92,93,90,92,...,89,89,88,87,89,80,85,87,65,88
JohnnyGaudreau,3,25,0,1,95,95,95,96,95,94,...,94,95,84,85,93,80,77,85,65,82
AlexOvechkin,3,33,1,1,90,96,90,95,80,96,...,88,88,95,88,88,89,91,90,80,94
ClaudeGiroux,0,30,1,0,93,93,94,94,85,93,...,91,91,84,85,91,82,82,85,70,84
JonathanHuberdeau,3,25,1,0,90,89,91,91,82,92,...,89,89,86,85,89,77,77,85,65,85


In [16]:
# knn model initatied and fitted
model = NearestNeighbors()
model.fit(df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [17]:
# get a list of the players
players = df.index.tolist()

In [18]:
# output player list, this is the set of players that can be fed into the model to find comparables
players

['AustonMatthews',
 'JohnnyGaudreau',
 'AlexOvechkin',
 'ClaudeGiroux',
 'JonathanHuberdeau',
 'MarcusJohansson',
 'MitchellMarner',
 'EliasPettersson',
 'GabrielLandeskog',
 'TeuvoTeravainen',
 'JakeDebrusk',
 'JakubVoracek',
 'VictorHedman',
 'EvgeniMalkin',
 'BoHorvat',
 'StevenStamkos',
 'MikkoRantanen',
 'TylerSeguin',
 'ArtemiPanarin',
 'BradMarchand',
 'AleksanderBarkov',
 'DavidPastrnak',
 'JohnTavares',
 'MarkStone',
 'DantonHeinen',
 'NinoNiederreiter',
 'DrewDoughty',
 'JamesVanRiemsdyk',
 'SeanMonahan',
 'EvgeniDadonov',
 'BraydenPoint',
 'ZachParise',
 'MattDuchene',
 'NikitaKucherov',
 'SebastianAho',
 'CamAtkinson',
 'JeffSkinner',
 'J.T.Miller',
 'AlexanderRadulov',
 'AndersLee',
 'JackEichel',
 'DylanLarkin',
 'WilliamNylander',
 'EvgenyKuznetsov',
 'ChrisTierney',
 'NathanMackinnon',
 'PhilKessel',
 'NazemKadri',
 'SidneyCrosby',
 'BrockBoeser',
 'RyanNugent-Hopkins',
 'MarkScheifele',
 'EliasLindholm',
 'TysonBarries',
 'VladimirTarasenko',
 'AlexDebrincat',
 'Connor

In [21]:
# player we want the comparables for
playerKey = 'MitchellMarner'

In [22]:
# get the comparables for the player above
comparables = model.kneighbors([df.loc[playerKey,:]],4,False)
comparables

array([[ 6, 21, 43,  4]])

In [23]:
# convert the array to a list
comparables = list(comparables[0])
comparables

[6, 21, 43, 4]

In [24]:
# get the attributes of the comparable players
comparablesAttributes = []
for i in comparables:
    comparablesAttributes.append(df.iloc[i])
comparablesAttributes[1:]

[Position                2
 Age                    22
 Shoots                  1
 Player Type             0
 Deking                 94
 Hand-Eye               92
 Passing                92
 Puck Control           93
 Discipline             83
 Off. Awareness         92
 Poise                  90
 Slap Shot Accuracy     92
 Slap Shot Power        88
 Wrist Shot Accuracy    93
 Wrist Shot Power       88
 Def. Awareness         89
 Faceoffs               68
 Shot Blocking          78
 Stick Checking         88
 Acceleration           91
 Agility                92
 Balance                85
 Endurance              84
 Speed                  91
 Aggressiveness         80
 Body Checking          80
 Durability             85
 Fighting Skill         65
 Strength               84
 Name: DavidPastrnak, dtype: int64, Position                0
 Age                    26
 Shoots                  0
 Player Type             0
 Deking                 95
 Hand-Eye               92
 Passing            

In [25]:
# create a dataframe for the comparables and their attributes
comp = pd.DataFrame()
for i in comparables:
    comp = comp.append(df.iloc[i,:])
    comp['Key'] = comp.index
comp

,Acceleration,Age,Aggressiveness,Agility,Balance,Body Checking,Def. Awareness,Deking,Discipline,Durability,...,Shoots,Shot Blocking,Slap Shot Accuracy,Slap Shot Power,Speed,Stick Checking,Strength,Wrist Shot Accuracy,Wrist Shot Power,Key
MitchellMarner,90.0,21.0,81.0,92.0,84.0,77.0,90.0,93.0,85.0,85.0,...,1.0,78.0,91.0,87.0,90.0,90.0,82.0,91.0,87.0,MitchellMarner
DavidPastrnak,91.0,22.0,80.0,92.0,85.0,80.0,89.0,94.0,83.0,85.0,...,1.0,78.0,92.0,88.0,91.0,88.0,84.0,93.0,88.0,DavidPastrnak
EvgenyKuznetsov,93.0,26.0,80.0,93.0,85.0,78.0,89.0,95.0,85.0,85.0,...,0.0,75.0,92.0,89.0,93.0,87.0,85.0,93.0,88.0,EvgenyKuznetsov
JonathanHuberdeau,89.0,25.0,77.0,89.0,86.0,77.0,89.0,90.0,82.0,85.0,...,1.0,81.0,88.0,88.0,89.0,89.0,85.0,88.0,88.0,JonathanHuberdeau


In [26]:
# convert index to keys
comparableKey = list(comp['Key'])
comparableKey

['MitchellMarner', 'DavidPastrnak', 'EvgenyKuznetsov', 'JonathanHuberdeau']

Based off the the NHL 19 attribute data, this suggests that Mitch Marner's comparables are: David Pastrnak, Evgeny Kuznetsov and, Jonathan Huberdeau.

Let's add in some salary data to determine what range Marner should fall into for his next contract.

In [27]:
# to include salary, load salary data obtained from capfriendly
salary = pd.read_csv('nhlplayersalary.csv')

In [28]:
# take a look at the salary data
salary.head()

,Key,PLAYER,TEAM,AGE,POS,HANDED,GP,G,A,P,...,LENGTH,EXPIRY,EXP. YEAR,CAP HIT,CAP HIT %,AAV,SALARY,BASE SALARY,S.BONUS,P.BONUS
0,ConnorMcDavid,Connor McDavid,EDM,21,C,Left,77,41,75,116,...,8,UFA,2026,"$12,500,000",16.7%,"$12,500,000","$15,000,000","$2,000,000","$13,000,000",$0
1,JohnTavares,John Tavares,TOR,27,C,Left,81,47,41,88,...,7,UFA,2025,"$11,000,000",13.8%,"$11,000,000","$15,900,000","$650,000","$15,250,000",$0
2,JonathanToews,Jonathan Toews,CHI,30,C,Left,81,35,46,81,...,8,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$12,000,000","$6,000,000","$6,000,000",$0
3,PatrickKane,Patrick Kane,CHI,29,"RW, C",Left,80,44,65,109,...,8,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$12,000,000","$6,000,000","$6,000,000",$0
4,CareyPrice,Carey Price,MTL,30,G,Left,66,-,-,-,...,8,UFA,2026,"$10,500,000",14.0%,"$10,500,000","$15,000,000","$2,000,000","$13,000,000",$0


In [29]:
# take a look at the table info
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 32 columns):
Key             984 non-null object
PLAYER          984 non-null object
TEAM            984 non-null object
AGE             984 non-null int64
POS             984 non-null object
HANDED          984 non-null object
GP              984 non-null int64
G               984 non-null object
A               984 non-null object
P               984 non-null object
P/GP            984 non-null object
+/-             984 non-null object
Sh              984 non-null object
Sh%             984 non-null object
W               984 non-null object
L               984 non-null object
SO              984 non-null object
GAA             984 non-null object
Sv%             984 non-null object
SIGNING DATE    984 non-null object
SIGNING AGE     984 non-null int64
CLAUSE          984 non-null object
LENGTH          984 non-null int64
EXPIRY          984 non-null object
EXP. YEAR       984 non-null int64


In [30]:
# set the index as the key
salary.set_index('Key', inplace=True)
salary.head()

,PLAYER,TEAM,AGE,POS,HANDED,GP,G,A,P,P/GP,...,LENGTH,EXPIRY,EXP. YEAR,CAP HIT,CAP HIT %,AAV,SALARY,BASE SALARY,S.BONUS,P.BONUS
Key,,,,,,,,,,,,,,,,,,,,,
ConnorMcDavid,Connor McDavid,EDM,21,C,Left,77,41,75,116,1.51,...,8,UFA,2026,"$12,500,000",16.7%,"$12,500,000","$15,000,000","$2,000,000","$13,000,000",$0
JohnTavares,John Tavares,TOR,27,C,Left,81,47,41,88,1.09,...,7,UFA,2025,"$11,000,000",13.8%,"$11,000,000","$15,900,000","$650,000","$15,250,000",$0
JonathanToews,Jonathan Toews,CHI,30,C,Left,81,35,46,81,1.00,...,8,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$12,000,000","$6,000,000","$6,000,000",$0
PatrickKane,Patrick Kane,CHI,29,"RW, C",Left,80,44,65,109,1.36,...,8,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$12,000,000","$6,000,000","$6,000,000",$0
CareyPrice,Carey Price,MTL,30,G,Left,66,-,-,-,-,...,8,UFA,2026,"$10,500,000",14.0%,"$10,500,000","$15,000,000","$2,000,000","$13,000,000",$0


In [31]:
# drop columns that we don't intend to use
salary.drop(columns=['AGE','POS','HANDED','W','L','SO','GAA','Sv%','PLAYER'], inplace = True)
salary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984 entries, ConnorMcDavid to KailerYamamoto
Data columns (total 22 columns):
TEAM            984 non-null object
GP              984 non-null int64
G               984 non-null object
A               984 non-null object
P               984 non-null object
P/GP            984 non-null object
+/-             984 non-null object
Sh              984 non-null object
Sh%             984 non-null object
SIGNING DATE    984 non-null object
SIGNING AGE     984 non-null int64
CLAUSE          984 non-null object
LENGTH          984 non-null int64
EXPIRY          984 non-null object
EXP. YEAR       984 non-null int64
CAP HIT         984 non-null object
CAP HIT %       984 non-null object
AAV             984 non-null object
SALARY          984 non-null object
BASE SALARY     984 non-null object
S.BONUS         984 non-null object
P.BONUS         984 non-null object
dtypes: int64(4), object(18)
memory usage: 176.8+ KB


In [32]:
# create a table for the salaries of the comparable players
compSalary = pd.DataFrame()
for i in comparableKey:
    compSalary = compSalary.append(salary.loc[i,:])
compSalary.head()

,+/-,A,AAV,BASE SALARY,CAP HIT,CAP HIT %,CLAUSE,EXP. YEAR,EXPIRY,G,...,P,P.BONUS,P/GP,S.BONUS,SALARY,SIGNING AGE,SIGNING DATE,Sh,Sh%,TEAM
MitchellMarner,20,67,"$1,744,167","$832,500","$894,167",1.3%,None,2019.0,RFA,26,...,93,"$850,000",1.15,$0,"$832,500",18.0,"Jul 28, 2015",231,0.11,TOR
DavidPastrnak,7,41,"$6,666,666","$8,000,000","$6,666,666",8.9%,None,2023.0,UFA,38,...,79,$0,1.22,$0,"$8,000,000",21.0,"Sep 14, 2017",233,0.16,BOS
EvgenyKuznetsov,9,51,"$7,800,000","$5,000,000","$7,800,000",10.4%,None,2025.0,UFA,21,...,72,$0,0.96,"$5,000,000","$10,000,000",25.0,"Jul 2, 2017",193,0.11,WSH
JonathanHuberdeau,-15,61,"$5,900,000","$6,200,000","$5,900,000",8.1%,None,2023.0,UFA,29,...,90,$0,1.11,$0,"$6,200,000",23.0,"Sep 6, 2016",220,0.13,FLA


In [33]:
# look at the infor about the comparable salary
compSalary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, MitchellMarner to JonathanHuberdeau
Data columns (total 22 columns):
+/-             4 non-null object
A               4 non-null object
AAV             4 non-null object
BASE SALARY     4 non-null object
CAP HIT         4 non-null object
CAP HIT %       4 non-null object
CLAUSE          4 non-null object
EXP. YEAR       4 non-null float64
EXPIRY          4 non-null object
G               4 non-null object
GP              4 non-null float64
LENGTH          4 non-null float64
P               4 non-null object
P.BONUS         4 non-null object
P/GP            4 non-null object
S.BONUS         4 non-null object
SALARY          4 non-null object
SIGNING AGE     4 non-null float64
SIGNING DATE    4 non-null object
Sh              4 non-null object
Sh%             4 non-null object
TEAM            4 non-null object
dtypes: float64(4), object(18)
memory usage: 736.0+ bytes


In [34]:
# reorder the columns 
compSalary = compSalary[['GP','G','A','P','P/GP','+/-','Sh','Sh%','AAV','CAP HIT', 'CAP HIT %','CLAUSE','EXP. YEAR','LENGTH','SIGNING AGE','TEAM']]

In [35]:
# view the table
compSalary

,GP,G,A,P,P/GP,+/-,Sh,Sh%,AAV,CAP HIT,...,SALARY,P.BONUS,S.BONUS,CLAUSE,EXP. YEAR,EXPIRY,LENGTH,SIGNING AGE,SIGNING DATE,TEAM
MitchellMarner,81.0,26,67,93,1.15,20,231,0.11,"$1,744,167","$894,167",...,"$832,500","$850,000",$0,None,2019.0,RFA,3.0,18.0,"Jul 28, 2015",TOR
DavidPastrnak,65.0,38,41,79,1.22,7,233,0.16,"$6,666,666","$6,666,666",...,"$8,000,000",$0,$0,None,2023.0,UFA,6.0,21.0,"Sep 14, 2017",BOS
EvgenyKuznetsov,75.0,21,51,72,0.96,9,193,0.11,"$7,800,000","$7,800,000",...,"$10,000,000",$0,"$5,000,000",None,2025.0,UFA,8.0,25.0,"Jul 2, 2017",WSH
JonathanHuberdeau,81.0,29,61,90,1.11,-15,220,0.13,"$5,900,000","$5,900,000",...,"$6,200,000",$0,$0,None,2023.0,UFA,6.0,23.0,"Sep 6, 2016",FLA


In [36]:
# join in the attribute data used to generate the comparables
compSalary = compSalary.join(comp)
compSalary

,GP,G,A,P,P/GP,+/-,Sh,Sh%,AAV,CAP HIT,...,Shoots,Shot Blocking,Slap Shot Accuracy,Slap Shot Power,Speed,Stick Checking,Strength,Wrist Shot Accuracy,Wrist Shot Power,Key
MitchellMarner,81.0,26,67,93,1.15,20,231,0.11,"$1,744,167","$894,167",...,1.0,78.0,91.0,87.0,90.0,90.0,82.0,91.0,87.0,MitchellMarner
DavidPastrnak,65.0,38,41,79,1.22,7,233,0.16,"$6,666,666","$6,666,666",...,1.0,78.0,92.0,88.0,91.0,88.0,84.0,93.0,88.0,DavidPastrnak
EvgenyKuznetsov,75.0,21,51,72,0.96,9,193,0.11,"$7,800,000","$7,800,000",...,0.0,75.0,92.0,89.0,93.0,87.0,85.0,93.0,88.0,EvgenyKuznetsov
JonathanHuberdeau,81.0,29,61,90,1.11,-15,220,0.13,"$5,900,000","$5,900,000",...,1.0,81.0,88.0,88.0,89.0,89.0,85.0,88.0,88.0,JonathanHuberdeau


In [37]:
# look at the info of the joined table
compSalary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, MitchellMarner to JonathanHuberdeau
Data columns (total 52 columns):
GP                     4 non-null float64
G                      4 non-null object
A                      4 non-null object
P                      4 non-null object
P/GP                   4 non-null object
+/-                    4 non-null object
Sh                     4 non-null object
Sh%                    4 non-null object
AAV                    4 non-null object
CAP HIT                4 non-null object
CAP HIT %              4 non-null object
BASE SALARY            4 non-null object
SALARY                 4 non-null object
P.BONUS                4 non-null object
S.BONUS                4 non-null object
CLAUSE                 4 non-null object
EXP. YEAR              4 non-null float64
EXPIRY                 4 non-null object
LENGTH                 4 non-null float64
SIGNING AGE            4 non-null float64
SIGNING DATE           4 non-null object
TEAM      

In [38]:
# select the salary range attributes to look at
compSalary.loc[:,'AAV':]

,AAV,CAP HIT,CAP HIT %,BASE SALARY,SALARY,P.BONUS,S.BONUS,CLAUSE,EXP. YEAR,EXPIRY,LENGTH,SIGNING AGE,SIGNING DATE
MitchellMarner,"$1,744,167","$894,167",1.3%,"$832,500","$832,500","$850,000",$0,None,2019.0,RFA,3.0,18.0,"Jul 28, 2015"
DavidPastrnak,"$6,666,666","$6,666,666",8.9%,"$8,000,000","$8,000,000",$0,$0,None,2023.0,UFA,6.0,21.0,"Sep 14, 2017"
EvgenyKuznetsov,"$7,800,000","$7,800,000",10.4%,"$5,000,000","$10,000,000",$0,"$5,000,000",None,2025.0,UFA,8.0,25.0,"Jul 2, 2017"
JonathanHuberdeau,"$5,900,000","$5,900,000",8.1%,"$6,200,000","$6,200,000",$0,$0,None,2023.0,UFA,6.0,23.0,"Sep 6, 2016"


Looking at Marner's comparables, they all fall within a 8-10.5\% as a cap hit percentage for their team. The current projection for the 2019-2020 season for salary cap is $83 million. This would suggest Marner's range is between 6.64 and 8.715 million AAV. Looking at their production from the compSalary table, we can see that Marner's overall point production is the highest of the 4. We can also see that Kuznetsov and Marner had the lowest shooting percentage. This would suggest that Pastrnak and Huberdeau are either better shooters or had more luck when it came to goal scoring. Marner's P/GP is second behind Pastrnak and that he scored less goals. This makes the Marner/Pastrnak comparison interesting as goal scoring is considered more valuable than accruing assists. Huberdeau is the closest in terms of overall production, with 90 points to Marner's 93. He also has scored more goals by 3, which could weigh their production at about even. Kuznetsov is an outlier within this comparasion. He has the fewest points, lowest scoring rate and has the highest salary. One explanation for this could be that he is a center whilst the other three are wingers. The general prevailing trend is the centers tend to get paid more than wingers. This makes sense as the responsibilities for the centers are higher and there are fewer high end centers than there are wingers. The lower supply and their higher importance could be used to explain the salary increase. Another potential explanation is that he signed his contract at an older age. This eats up UFA (unrestricted free agent) years, when other teams are free to negotiate and offer contracts during the free agent period without owing his current team anything. This means that he is sacrificing the potential to be offered more on the open market, hence getting a premium on his current deal to address that risk. This would suggest that when incorporating production with the attributes, it is more likely that Marner's closest comparables are Pastrnak and Huberdeau and should be around 8-9\% of the salary cap. This is well off what the media are suggesting he should get for his next contract. The media reports have him making between 9-11 million AAV which would be 10.8-13.2\% of the salary cap.

## Issues with Analysis

Looking at our demonstration, there are a couple of issues. 

The first issue is that the attributes that we are using are not reliable and we have only taken a subset of players from the game. We have pulled them from the NHL 19 video game and this proxy data has no real value. We do not know how Electronic Arts has determined the ratings for the attributes. They could be basing it on real factual data based off of historical NHL games or they could simply be making it up. We have also sampled it to only include 250 players, this is about 25% of the players in the league. This gives a chance that there are better comparables that have simply not been considered by the model. To improve this going forward, we should utilize real stats and design a set of input variables that take into account the: eye-test, production, and advanced analytics.

The second issue that we have here, is that we are using a lot of attributes for the kNN analysis. We fed the model just under 30 variables with a small dataset, this provides the risk of overfitting. If we were to repeat this experiment we would be better off feature engineering in order to decrease the number of inputs in the model. For example, you could combine both slap shot power and wrist shot power together and generate one input for shot power. Or you could combine speed, acceleration and agility to form one metric for speed. You could combine the physical traits (body checking, aggressiveness, and strength) into a single metric. Another option would be to accumulate the statistics for all of the players.

The third issue with this analysis is that we do not have an accuracy test. How do we know that the three players identified are actually comparables for the player we tested. If we are using the Marner example, none of the comparables we generated are often cited in the media as comparison points. Furthermore, there is no data that we could use directly to compare too. To see if we are in the right ballpark, we would have to identify a set of comparables generated by hockey analysts or scouts or other models and see if our comparables are within the set. 

The last issue that needs to be addressed is that this does not take into account how players vary over time. It doesn't take into account how the salary cap has grown over time, their importance to the team, how their production has increased over time, how their skills have improved over time, their historical production, and age curves. For a demonstration is not neccessary, it is critical to point out. A player's production in their mid to late 20's will in all likelihood differ from their production in their early to mid 30's. Furthermore, some of their skills might regress over time relative to the competition. This changes their impact on the game. When signing a player to a long term contract forecasting how they are going to perform down the road becomes important. It is not unheard of a player being signed to a long-term deal and a couple years in, no longer being worth it for the team that signed them. Taking to account salary considerations and player's impact over time is a very real concern for NHL teams.

If we were to extend this analysis further, these are some of the issues that we could address. Steps to be taken include: obtain real data utilizing the NHL API and feature engineering, restrict input variables, identify externally generated comparables, and take into account the time series nature of the data.

## Conclusion

Earlier, we indicated that there are two situations a team could use the comparables generated: for salary negotiations and for team construction. For the salary example we have explored earlier and to use the model, the main change that would need to be made is changing the input variables to match the metrics relevant for comparison. This comes down to the philsophy behind how a General Manager wants to build their team and which factors they deem most important. The kNN model could take these input metrics along with the player that is seeking a new contract and find their closest comparables on the metrics identified as being the most important indicators of success. Team construction would be the same. You could use the model to either find replacements for the player you lost or you could find players similar or slightly better in areas you want to improve or you could find players adjacent to ones you may want to acquire but are unable to. The kNN model can be used as a tool in order to help management come up with better comparables and improve their ability to maximize the value they can get out of the resources that they have.